<a href="https://colab.research.google.com/github/41371130H/PL-Repo/blob/main/HW2_%E6%97%A5%E5%B8%B8%E6%94%AF%E5%87%BA%E9%80%9F%E7%AE%97%E8%88%87%E5%88%86%E6%94%A4%2BAI%E5%9B%9E%E9%A5%8B(Gradio%E8%B6%85%E6%99%82).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **操作說明**
  有三種動作可供選擇



---



##   **新增資料**

  ***日期時間自動輸入***
  1.  輸入項目（1: 早餐, 2: 午餐, 3: 晚餐, 4: 飲料, 5: 交通, 6: 就學用品, 7: 日常用品）
  2.  輸入品項
  3.  輸入金額
  4.  是否要AA分攤（yes or no）

          yes：自動除以輸入之人數並顯示實付金額在輸出區與試算表


---


##   **刪除資料**

依照提供之表格輸入要刪除的資料列號


---


##   **統整資料**

Gemini的分析回饋

    ***由於Gradio的時間限制 or Gemini API的問題，此為超時的Gradio的版本，敬請見諒***



---



**Google Sheet**：https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing







In [1]:
!pip install -q gspread google-auth google-generativeai
import google.generativeai as genai
from google.colab import auth, userdata
auth.authenticate_user()
import gspread
from google.auth import default
creds, _ = default()
gc = gspread.authorize(creds)

import pandas as pd
import datetime
import pytz
import gradio as gr
import ipywidgets as widgets
from IPython.display import display, clear_output

# ========== 設定 Gemini ==========
genai.configure(api_key=userdata.get("gemini"))
model = genai.GenerativeModel("gemini-1.5-flash")

# 連線到 Google Sheet
gsheets = gc.open_by_url('https://docs.google.com/spreadsheets/d/1KhaWfYSh2fGRmwC8SkZG8GdbTshYql4QX-hknk0ON84/edit?usp=sharing')
worksheet = gsheets.worksheet('工作表1')
sheet2 = gsheets.worksheet('工作表2')   # 🔹 新增連線到工作表2

In [2]:
# 超時
from google.oauth2.service_account import Credentials
# ========== 功能 1：新增資料 ==========
def add_record(category, item, amount, aa, people):
    tz = pytz.timezone("Asia/Taipei")
    now = datetime.datetime.now(tz)
    date_str = now.strftime("%Y-%m-%d")
    time_str = now.strftime("%H:%M")

    # 是否AA分攤
    if aa == "yes" and people > 0:
        amount = amount // people

    new_data = pd.DataFrame([{
        "日期": date_str,
        "時間": time_str,
        "項目": category,  # 直接用選項文字
        "品項": item,
        "金額": amount
    }])

    data_to_write = new_data.values.tolist()
    worksheet.append_rows(values=data_to_write, value_input_option="USER_ENTERED")

    return f"✅ 已新增資料：{category} - {item} - {amount} 元", show_records()


# ========== 功能 2：顯示 / 刪除資料 ==========
def show_records():
    all_data = worksheet.get_all_values()
    df = pd.DataFrame(all_data[1:], columns=all_data[0])
    df["金額"] = pd.to_numeric(df["金額"], errors="coerce")
    # 加入列號欄位
    return df.reset_index(drop=True).reset_index().rename(columns={"index": "列號"})

def delete_record(row_number):
    try:
        worksheet.delete_rows(int(row_number) + 2)
        return f"🗑️ 已刪除第 {row_number} 列資料", show_records()
    except Exception as e:
        return f"❌ 刪除失敗: {e}", show_records()

def analyze_records():
    all_data = worksheet.get_all_values()
    if not all_data or len(all_data) < 2:
        return "⚠️ 沒有資料可以分析"

    df = pd.DataFrame(all_data[1:], columns=all_data[0])
    prompt = f"""
    以下是我的收支表：
    {df.to_string(index=False)}

    請根據資料，給我收支的總結與三點具體建議，
    包含：花費最多的項目、可能的節省方向、以及理財提醒。
    """

    try:
        response = model.generate_content(prompt, request_options={"timeout": 10})
        advice = response.text if response.text else "⚠️ Gemini 沒有回傳內容"
    except Exception as e:
        advice = f"❌ Gemini API 發生錯誤: {e}"

    # 寫入工作表2
    tz = pytz.timezone("Asia/Taipei")
    timestamp = datetime.datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    try:
        sheet2.append_row([timestamp, advice])
    except Exception as e:
        advice += f"\n⚠️ 無法寫入工作表2: {e}"

    # 🔹 從工作表2抓最新回覆（確保顯示的是存入的版本）
    result_data = sheet2.get_all_values()
    if len(result_data) > 1:
        latest_result = result_data[-1][1]  # 第二欄 = 分析內容
    else:
        latest_result = advice  # 如果沒存成功就直接顯示 Gemini 結果

    return latest_result





# ========== Gradio 介面 ==========
with gr.Blocks() as demo:
    gr.Markdown("# 💵 Google Sheet 記帳系統")

    # 📋 紀錄表格（全域元件）
    records_table = gr.Dataframe(
        value=show_records(),
        label="📋 目前紀錄",
        interactive=False
    )

    with gr.Tab("新增資料"):
        category = gr.Dropdown(
            ["早餐", "午餐", "晚餐", "飲料", "交通", "就學用品", "日常用品", "其他"],
            label="項目"
        )
        item = gr.Textbox(label="品項")
        amount = gr.Number(label="金額", precision=0)
        aa = gr.Radio(["yes", "no"], label="是否AA分攤", value="no")
        people = gr.Number(label="分攤人數", value=1, precision=0, visible=False)
        add_btn = gr.Button("新增資料")
        add_output = gr.Textbox(label="結果")

        # 🔹 控制分攤人數欄位顯示/隱藏
        def toggle_people(aa_choice):
            if aa_choice == "yes":
                return gr.update(visible=True, value=1)
            else:
                return gr.update(visible=False, value=1)

        aa.change(fn=toggle_people, inputs=aa, outputs=people)

        add_btn.click(
            fn=add_record,
            inputs=[category, item, amount, aa, people],
            outputs=[add_output, records_table]
        )

    with gr.Tab("刪除資料"):
        row_number = gr.Number(label="要刪除的列號 (從 0 開始)", precision=0)
        delete_btn = gr.Button("刪除資料")
        delete_output = gr.Textbox(label="結果")

        delete_btn.click(fn=delete_record, inputs=row_number, outputs=[delete_output, records_table])

    with gr.Tab("收支分析"):
        gr.Markdown("### 📊 Gemini 收支分析")
        analyze_btn = gr.Button("產生分析與建議")
        analyze_output = gr.Textbox(label="Gemini 分析結果", lines=10)

        analyze_btn.click(fn=analyze_records, inputs=None, outputs=analyze_output)

demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://36c1df48b7f3c61ca5.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
